# 99_Coverage_Report

In [ ]:
import json, io, inspect, platform, sys
from pathlib import Path
import numpy as np
import pandas as pd

PANDAS_VERSION = pd.__version__
PYTHON_VERSION = sys.version
PLATFORM_INFO = platform.platform()
print('pandas:', PANDAS_VERSION)
print('python:', PYTHON_VERSION)
print('platform:', PLATFORM_INFO)
if not PANDAS_VERSION.startswith('2.2.'):
    print('WARNING: pandas version != 2.2.*; inventory may differ.')


In [ ]:

inventory = json.loads(Path('inventory_pandas_api.json').read_text(encoding='utf-8'))
coverage_map = {
    'DataFrame': 'coverage_dataframe.json',
    'Series': 'coverage_series.json',
    'Index': 'coverage_index.json',
    'GroupBy': 'coverage_groupby.json',
    'Window': 'coverage_window.json',
    'TopLevel': 'coverage_toplevel.json',
}
all_missing = {}
all_unknown = {}
for target in inventory['meta']['target_order']:
    inv_records = inventory['targets'][target]
    inv_set = {r['qualname'] for r in inv_records}
    aliases = {r['qualname']: r['alias_of'] for r in inv_records if r.get('alias_of')}
    deprecated = [r['qualname'] for r in inv_records if r.get('deprecated') is True]
    cpath = Path(coverage_map[target])
    if cpath.exists():
        cov = json.loads(cpath.read_text(encoding='utf-8'))
    else:
        cov = {'covered_members': [], 'skipped_members': {}}
    covered = set(cov.get('covered_members', []))
    skipped = set(cov.get('skipped_members', {}).keys())
    unknown = sorted((covered | skipped) - inv_set)
    missing = sorted(inv_set - (covered | skipped))
    all_missing[target] = missing
    all_unknown[target] = unknown
    print('
===', target, '===')
    print('inventariati:', len(inv_set))
    print('covered:', len(covered))
    print('skipped:', len(skipped))
    print('MISSING:', len(missing))
    print('alias rilevati:', len(aliases))
    print('deprecati:', len(deprecated))
    print('unknown_coverage:', len(unknown))
    if missing:
        print('missing sample:', missing[:10])
    if unknown:
        print('unknown sample:', unknown[:10])

success = all(len(v) == 0 for v in all_missing.values()) and all(len(v) == 0 for v in all_unknown.values())
print('
SUCCESS:' if success else '
NOT YET SUCCESS:', success)
